In [1]:
# Loading Dependencies for Medical Image Analysis
import optuna
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt # Import matplotlib.pyplot
import torch.nn as nn
from torchsummary import summary
import torchvision.models as models
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import seaborn as sns
import torchvision
import torch.optim as optim  # Import the optim module
import numpy as np
np.bool = np.bool_
import torchvision.transforms as transforms
#from torch.utils.data import DataLoader, Subset
#from torchvision.datasets import ImageFolder
#from torchvision import datasets
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader, Subset
#from torchvision import transforms
import os
import torch
from PIL import Image
#import matplotlib.pyplot as plt
#import time
import copy
#import os
import IPython
import time
from sklearn.model_selection import KFold
import threading
import optuna.visualization
import albumentations as A
from albumentations.pytorch import ToTensorV2
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args
import GPyOpt
import GPy
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from torch.optim.lr_scheduler import (
    StepLR,
    ReduceLROnPlateau,
    CosineAnnealingLR,
    CyclicLR,
    OneCycleLR,
) 
import logging

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-30 13:28:36.669946: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-30 13:28:36.743313: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.8/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2

In [2]:
logging.basicConfig(
    filename='CNN_Model_Next_output.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    force=True  # ha notebookból futtatsz többször, ez kell
)

In [3]:
def create_scheduler(optimizer, scheduler_type, **kwargs): # This function defines the scheduler types
    if scheduler_type == "StepLR":
        return StepLR(optimizer, **kwargs)
    elif scheduler_type == "ReduceLROnPlateau":
        return ReduceLROnPlateau(optimizer, **kwargs)
    elif scheduler_type == "CosineAnnealingLR":
        return CosineAnnealingLR(optimizer, **kwargs)
    elif scheduler_type == "CyclicLR":
        return CyclicLR(optimizer, cycle_momentum=False, **kwargs)
    elif scheduler_type == "OneCycleLR":
        return OneCycleLR(optimizer, **kwargs)
    else:
        raise ValueError(f"Unknown scheduler type: {scheduler_type}")

In [4]:

def Model_ResNet101_PyTorch(learning_rate, dropout_rate):
    # Load pre-trained ResNet101
    model = models.resnet101(weights='ResNet101_Weights.DEFAULT')

    # Modify the input layer to accept 3 channels (assuming your input is RGB)
    # model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)

    # Freeze all layers initially
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze the desired layers for fine-tuning
    for param in model.layer3.parameters(): # unfreeze Layer 3
        param.requires_grad = True

    for param in model.layer4.parameters(): # unfreeze Layer 4
        param.requires_grad = True

    for param in model.fc.parameters(): # Unfreeze the fully connected layer
        param.requires_grad = True

    # Modification of the classifier layer (fully connected layer)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 3),        # 3 output classes (adjust if you have a different number of classes)
        nn.Dropout(dropout_rate),
        nn.Softmax(dim=1)              # Softmax activation for multi-class classification
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    summary(model, (3, 224, 224), device = str(device))  # Print model summary (adjust input size if needed)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss() # Appropriate for multi-class classification

    # Define optimizer - consider using a lower learning rate for earlier unfrozen layers if needed
    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate, weight_decay=0.01)


    return model, criterion, optimizer

In [5]:
best_params = {
    "learning_rate":0.00045710987419654515, #optimal learning rate from hyperparameter search
    "dropout_rate":0.25339643965539393,  # optimal dropout rate from hyperparameter search
    "batch_size":256,  # Your optimal batch size from hyperparameter search #########################################x/sry i have to modify original vas 64
    "num_epochs":83, #  optimal number of epochs from hyperparameter search
    #"factor":0.7324426614572568,  #  optimal factor from hyperparameter search
    #"patience":9, #  optimal patience from hyperparameter search
}

In [6]:
model, criterion, optimizer = Model_ResNet101_PyTorch(learning_rate=best_params["learning_rate"],
                                                      dropout_rate= best_params["dropout_rate"])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [7]:
class MedicalImageDataset(Dataset):
       def __init__(self, root_dir, image_extensions, transform=None):
           self.root_dir = root_dir
           self.image_extensions = image_extensions
           #self.transform = transform
           self.image_paths = []
           self.labels = []

           for class_name in os.listdir(root_dir):  
               class_path = os.path.join(root_dir, class_name)
               if os.path.isdir(class_path):
                   for filename in os.listdir(class_path):
                       if filename.lower().endswith(image_extensions):
                           self.image_paths.append(os.path.join(class_path, filename))
                           self.labels.append(class_name)  # Assuming subfolder name is the label

       def __len__(self):
           return len(self.image_paths)

       def __getitem__(self, idx):
           image_path = self.image_paths[idx]
           image = np.array(Image.open(image_path).convert('RGB'))  # Convert to NumPy array
           label = self.labels[idx]
           label_mapping = {'Benign': 0, 'Malignant': 1, 'Normal': 2}
           label = label_mapping.get(label)  # Apply the label mapping here
           #image = np.array(image)

           #if self.transform:
               #image = self.transform(image)

           return image, label

In [8]:
dataset_path = '/home/workspace/PetersWorkspace/Abbans/TMPFS_Medical_Images/Medical_Images/'
#dataset_path = '/usr/bin/Medical_Images'
image_extensions = ('.png', '.jpg', '.jpeg')

dataset = MedicalImageDataset(dataset_path, image_extensions)

In [ ]:
!ls /home/workspace/PetersWorkspace/Medical_Images/

In [9]:
# Splitting the dataset based on their split proportions
Train_ratio = 0.70 # Ration of Training Data
Validaton_ratio = 0.15  # Ratio of Testing Data
Testing_ratio = 0.15 # Ration of Testing Data

#torch.manual_seed(42)  # You can choose any integer as your seed
# Calculate the sizes of the train, validation, and test sets
Dataset_size = len(dataset) # Determines the dataset size
#Train_size = int(Train_ratio * Dataset_size) # Determines the train dataset size
#Validation_size = int(Validaton_ratio * Dataset_size) # Determines the validation dataset size
Test_size = int(Testing_ratio * Dataset_size) # Determines the test dataset size
#Test_size = Dataset_size - Train_size - Validation_size # Determines the test dataset size
Dataset_Train_Valid = Dataset_size - Test_size # Determines the new dataset size

# Split the dataset into train, validation, and test sets
 #Train_dataset, Validation_dataset, Test_dataset = torch.utils.data.random_split(dataset, [Train_size, Validation_size, Test_size])
Dataset_New, Test_dataset = torch.utils.data.random_split(dataset, [Dataset_Train_Valid, Test_size])

In [10]:
#print(len(Train_dataset))
#print(len(Validation_dataset))
#print(len(Test_dataset))
print(len(Dataset_New))
print(len(Test_dataset))

661
116


In [12]:
# Defining Augmentation for my Training Dataset
#For the training dataset
#For the training dataset
Fundamental_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.4),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomRotation(degrees=(-10, 10)),
    transforms.RandomAffine(degrees=0, translate=(0.08, 0.08), scale=(0.92, 1.08)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.85, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [15]:
Advanced_transform = A.Compose([
    A.MedianBlur(blur_limit=3, p=1),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=5, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.08, rotate_limit=12, p=0.5),
    #A.RandomSizedCrop(min_max_height=(160, 224), size =(224, 224), p=0.5),
    A.Resize(224, 224), # OR
    A.ElasticTransform(alpha=0.8, sigma=40, alpha_affine=40, p = 0.5),
    A.GridDistortion(p=0.4, distort_limit=0.4),
    A.OpticalDistortion(p=0.05, distort_limit=0.05, shift_limit=0.5),
    #A.GaussianBlur(blur_limit=(3, 7), p=0.5),  # Gaussian Blur applied after spatial transformations
    # CoarseDropout(Random Erasing)
    A.CoarseDropout(max_holes=1, max_height=32, max_width=32, min_holes=1, min_height=16,
                    min_width=16, fill_value=0,
        p=0.5,),

    # Pixel-Level Transformations
    A.RandomBrightnessContrast(brightness_limit=0.05, contrast_limit=0.05, p=0.3),
    #A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=20, p=0.4),
    A.CLAHE(clip_limit=3.0, tile_grid_size=(6, 6), p=0.25),  # Contrast Limited Adaptive Histogram Equalization

    # Noise Injection
    #A.GaussNoise(var_limit=(0.01, 0.05), p=0.5),
    #A.MultiplicativeNoise(multiplier=(0.9, 1.1), elementwise=True, p=0.3),  # Speckle noise

         # Blurring and Sharpening
    #A.OneOf([A.MedianBlur(blur_limit=3, p=0.5),],p=0.1),

    A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.5),

    # Normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [17]:
Train_transform = transforms.Compose([
    transforms.ToPILImage(), # Conversion of Images into PIL for Albumentation
    Advanced_transform, # Application of advanced transformations
    Fundamental_transform, # Application of fundamental transformation
])

In [18]:
# Defining Augmentation for my Validaton Dataset
Validation_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [19]:
# Defining Augmentation for the Testing Dataset
Test_transform = transforms.Compose([
    # Only resize and normalize for test set
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [20]:
# Original Training Model
def Train_Model(model, criterion, optimizer, dataloaders, dataset_sizes, num_epochs, scheduler, scheduler_type):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    #best_loss = float('inf')
    train_acc_history = []
    train_loss_history = []
    val_acc_history = []
    val_loss_history = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("GPU CPU? using device",device)
    #model.to(str(device))

    for epoch in range(num_epochs):
        #logging.info(f'Epoch {epoch}/{num_epochs - 1}') #
        print(f'Epoch {epoch}/{num_epochs - 1}')
        #logging.info('-' * 10) #
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()


                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() /dataset_sizes[phase]


            #print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model if it's the best validation accuracy so far
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())


            if phase == 'train':
               #logging.info(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}') #
               print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
               train_acc_history.append(epoch_acc.item())
               train_loss_history.append(epoch_loss)

            elif phase == 'val':
                #logging.info(f'Val Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}') #
                print(f'Val Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
                val_acc_history.append(epoch_acc.item())
                val_loss_history.append(epoch_loss)


        # Scheduler step logic based on scheduler_type
        if phase == 'val':
            if scheduler_type == "ReduceLROnPlateau":
              # Step the scheduler based on validation accuracy (negative because ReduceLROnPlateau minimizes)
              scheduler.step(-epoch_acc)
            elif scheduler_type in ["CosineAnnealingLR", "OneCycleLR", "CyclicLR"]:
              scheduler.step()
            elif scheduler_type == "StepLR":
              if (epoch + 1) % scheduler.step_size == 0:
                    scheduler.step()
                    # StepLR is typically called after a certain number of epochs (step_size)


    time_elapsed = time.time() - since
    #logging.info(f'Training is completed in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s') #
    print(f'Training is completed in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_acc

In [ ]:
import sys
from datetime import datetime

log = open("cell_output_BO.log", "a", buffering=1)  # line-buffered
sys.stdout = log
sys.stderr = log

now = datetime.now()
print("-------------------------------------------------------------------New Model objective running is started at : ", now, " -------------------------------------------------------------------------------------------------------")

# Objective Function for: Bayesian Optimization: Optuna
def objective(trial, K):
    import tensorflow as tf
    # We want to maximize accuracy, so we initialize with a low value
    best_acc = 0.0
    import gc  # Add this line to import the gc module
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Define the search space for hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-2, log=True)
    dropout_rate = trial.suggest_float("dropout_rate", 0.05, 0.5)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    num_epochs = trial.suggest_int("num_epochs", 10, 100)
    gamma1 = trial.suggest_float("gamma1", 1e-3, 1e1, log=True)  # Range: 0.001 to 10.0

     # Scheduler lr types:
    scheduler_type = trial.suggest_categorical(
        "scheduler",
        [
            "StepLR",
            "ReduceLROnPlateau",
            "CosineAnnealingLR",
            "CyclicLR",
            "OneCycleLR",
        ],
    )
    # Initialize lists to store validation accuracies for each fold
    fold_val_accuracies = []

    #Dataset_New = MedicalImageDataset(dataset_path, image_extensions)

    # Create KFold object
    kf = KFold(n_splits=K, shuffle=True, random_state=42)

    # Iterate through folds
    for fold, (train_idx, val_idx) in enumerate(kf.split(Dataset_New)):
       print(f"Fold {fold + 1}/{K}")

        # Create train and validation subsets
        Train_subset = Subset(Dataset_New, train_idx) # Create subset for training for training dataset
        Val_subset = Subset(Dataset_New, val_idx) # Create subsef for validation dataset

        print("Train_subset:", len(Train_subset))
        print("Val_subset:", len(Val_subset))


        Train_subset.dataset.transform = Train_transform # Transformation for training
        Val_subset.dataset.transform = Validation_transform # Transformaing for Validation
        # Create data loaders for train and validation subsets

        # Define custom_collate_fn here, inside the objective function
        def custom_collate_fn(batch):
            images = []
            labels = []

            for image, label in batch:
                image = transforms.ToPILImage()(image)  # Convert to PIL Image
                image = transforms.Resize((224, 224))(image)  # Add resize transform here
                image = transforms.ToTensor()(image)  # Convert to Tensor using ToTensor
                images.append(image)
                labels.append(label)

            # Stack images into a batch tensor
            images = torch.stack(images, dim=0)
            labels = torch.tensor(labels)

            return images, labels

        train_loader = DataLoader(Train_subset, batch_size=batch_size,num_workers=32, shuffle=True,collate_fn=custom_collate_fn)
        val_loader = DataLoader(Val_subset, batch_size=batch_size,num_workers=32, shuffle=False, collate_fn=custom_collate_fn ) #modified num_workers!

        dataloaders = {
            'train': train_loader,
            'val': val_loader
        }

        dataset_sizes = {
            'train': len(Train_subset),
            'val': len(Val_subset)
        }
        print("Steps_Per_Epochs", len(dataloaders["train"]))
      # Create a new model with the suggested hyperparameters
        model, criterion, optimizer = Model_ResNet101_PyTorch(learning_rate=learning_rate, dropout_rate=dropout_rate)

        # Create Scheduler based on scheduler_type with conditional logic within objective function
        scheduler_kwargs = {}
        if scheduler_type == "StepLR":
            scheduler_kwargs["step_size"] = trial.suggest_int("step_size", 1, 10)
            scheduler_kwargs["gamma"] = trial.suggest_float("gamma", 0.1, 0.9)
        elif scheduler_type == "ReduceLROnPlateau":
            scheduler_kwargs["factor"] = trial.suggest_float("factor", 0.1, 0.9)
            scheduler_kwargs["patience"] = trial.suggest_int("patience", 1, 10)
        elif scheduler_type == "CosineAnnealingLR":
            scheduler_kwargs["T_max"] = trial.suggest_int("T_max", 1, 10)
            scheduler_kwargs["eta_min"] = trial.suggest_float("eta_min", 1e-6, 1e-4)
        elif scheduler_type == "CyclicLR":
            scheduler_kwargs["base_lr"] = trial.suggest_float("base_lr", 1e-6, 1e-4)
            scheduler_kwargs["max_lr"] = trial.suggest_float("max_lr", 1e-4, 1e-1)
            scheduler_kwargs["step_size_up"] = trial.suggest_int("step_size_up", 1, 10)
            scheduler_kwargs["step_size_down"] = trial.suggest_int("step_size_down", 1, 10)
            scheduler_kwargs["mode"] = trial.suggest_categorical("mode", ["triangular", "triangular2", "exp_range"])
        elif scheduler_type == "OneCycleLR":
            scheduler_kwargs["max_lr"] = trial.suggest_float("max_lr", 1e-4, 1e-1)
            scheduler_kwargs["epochs"] = num_epochs  # Use the total number of epochs
            scheduler_kwargs["steps_per_epoch"] = len(dataloaders["train"])

        scheduler = create_scheduler(optimizer, scheduler_type, **scheduler_kwargs)
    # Train the model and get the validation accuracy
        _, val_acc = Train_Model(model, criterion, optimizer,
                                  dataloaders,
                                  dataset_sizes,
                                  num_epochs=num_epochs, scheduler= scheduler, scheduler_type = scheduler_type)
        fold_val_accuracies.append(val_acc)
    # Calculate objective function value based on validation accuracy
    mean_val_accuracy = sum(fold_val_accuracies) / K if K > 0 else 0
    val_accuracy_variance = sum([(acc - mean_val_accuracy) ** 2 for acc in fold_val_accuracies]) / K if K > 0 else 0


    # We want to maximize accuracy, so we minimize the negative accuracy,
    # incorporating the variance as a penalty (minimizing variance)
    objective_value = -mean_val_accuracy + gamma1 * val_accuracy_variance

    allocated = torch.cuda.memory_allocated(device) / (1024 ** 2)  # MB
    reserved = torch.cuda.memory_reserved(device) / (1024 ** 2)    # MB
    print(f"[CUDA] Allocated Memory: {allocated:.2f} MB, Reserved Memory: {reserved:.2f} MB")
    del model
    del criterion
    del optimizer
    del scheduler
    del dataloaders
    del dataset_sizes
    del Train_subset
    del Val_subset
    del train_loader
    del val_loader
    gc.collect()
    torch.cuda.empty_cache()
    tf.keras.backend.clear_session()
    return objective_value.item() # Converts from GPU to CPU

K = 3 # Change the value of K here after the training process is completed
# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction="minimize")  # Minimize the objective value (which is negative accuracy)
study.optimize(lambda trial: objective(trial, K), n_trials=50)  # Adjust n_trials as needed

# Print the best hyperparameters and the corresponding validation accuracy
print(f"Best Hyperparameters for K = {K}:", study.best_params)
# Since we minimized negative accuracy, the best value is the negative of the best accuracy
print(f"Best Validation Accuracy for K = {K}:", -study.best_value)